# Part IV — Professional Django  
## 20. Performance and Scaling Fundamentals (Measure → Optimize → Verify)

Performance is not “guess and tweak.” Industry-standard performance work is a loop:

1. **Measure** (profiling, query counts, timing)
2. **Identify bottlenecks** (DB queries, N+1, slow templates, serialization)
3. **Fix** using known patterns (ORM optimization, indexes, caching)
4. **Verify** with tests/benchmarks and monitor in production

This chapter gives you a professional toolkit for Django performance at the app
level (not only infrastructure).

---

## 20.0 Learning Outcomes

By the end of this chapter you should be able to:

1. Measure performance:
   - per-request timing
   - query counts and slow queries
   - basic profiling
2. Identify and fix:
   - N+1 query problems
   - missing indexes and slow filters/orderings
   - expensive template loops
3. Use the right ORM tools:
   - `select_related`, `prefetch_related`, `Prefetch`
   - `annotate`, `exists`, `values`
4. Use caching correctly:
   - per-view caching
   - template fragment caching
   - low-level caching
   - cache invalidation patterns
5. Add pagination and understand its scaling limits (offset vs cursor concepts).
6. Build a performance regression test using query counts.
7. Create a production performance checklist.

---

## 20.1 Performance Baselines: What “Fast Enough” Means

Different apps have different goals, but a common baseline for server-rendered pages:

- 95th percentile response time under typical load: < 200–500ms for most pages
- DB query count on list/detail pages: usually small and stable (often < 10; ideally
  2–5 for many pages)
- Avoid huge response bodies
- Avoid unbounded queries (always paginate lists)

**Rule:** optimize user-visible paths first:
- article list/detail
- tasks list/detail
- login
- admin changelists (internal productivity)

---

## 20.2 Measuring: You Can’t Optimize What You Don’t Measure

### 20.2.1 Add request timing header (you already did)
If you have `X-Request-Duration-Ms`, you already have a coarse measurement.

Use it to compare before/after changes.

### 20.2.2 Django Debug Toolbar (dev-only, industry common)

#### Install
Add to `requirements-dev.txt`:

```text
django-debug-toolbar
```

Install:

```bash
python -m pip install -r requirements-dev.txt
```

#### Add to settings (dev only)
In `config/settings.py` (or `config/settings/dev.py` if you split settings):

```python
INSTALLED_APPS += ["debug_toolbar"]
MIDDLEWARE = ["debug_toolbar.middleware.DebugToolbarMiddleware"] + MIDDLEWARE

INTERNAL_IPS = ["127.0.0.1"]
```

Add URL in `config/urls.py` (dev only):

```python
from django.conf import settings
from django.urls import include, path

if settings.DEBUG:
    urlpatterns = [
        path("__debug__/", include("debug_toolbar.urls")),
    ] + urlpatterns
```

#### What to look at in toolbar
- SQL panel:
  - number of queries
  - time per query
  - duplicate queries
- Templates panel:
  - template render time
- Cache panel (later)

**Why this is industry standard**
It makes N+1 issues and slow queries visible instantly.

---

## 20.3 The #1 Django Performance Problem: N+1 Queries

### 20.3.1 The N+1 pattern
You fetch a list of objects (1 query), then for each object fetch related objects (N queries).

Example (articles + tags):

```python
articles = Article.objects.published()
for a in articles:
    list(a.tags.all())
```

### 20.3.2 Fix with `prefetch_related` for M2M
```python
articles = Article.objects.published().prefetch_related("tags")
```

Now:
- one query for articles
- one query for tags
No per-article tag query.

### 20.3.3 Fix with `select_related` for FK
For tasks list, you show `assigned_to`, `created_by`.

Bad:

```python
tasks = Task.objects.filter(organization=org)
for t in tasks:
    t.assigned_to.username
```

Fix:

```python
tasks = Task.objects.filter(organization=org).select_related(
    "assigned_to", "created_by", "updated_by"
)
```

### 20.3.4 Verify with query-count tests
Use `assertNumQueries` or pytest’s `django_assert_num_queries`.

This prevents regressions when templates change.

---

## 20.4 QuerySet Efficiency: Use the Right Methods

### 20.4.1 Prefer `exists()` over loading objects
Bad:

```python
if Article.objects.filter(slug=slug):
    ...
```

Better:

```python
if Article.objects.filter(slug=slug).exists():
    ...
```

### 20.4.2 Prefer `count()` over `len(qs)`
Bad:

```python
len(Article.objects.published())
```

Better:

```python
Article.objects.published().count()
```

### 20.4.3 Use `values()` for lightweight responses
If you don’t need model instances, use `values()`:

```python
Article.objects.published().values("id", "slug", "title")
```

This can reduce overhead.

---

## 20.5 Indexes: The Most Underused Performance Tool in Django Apps

### 20.5.1 What an index does (practical explanation)
An index is a data structure that helps the database find rows quickly without scanning the entire table.

If you frequently query:
- by `slug`
- by `(organization, status, created_at)`
you should index those fields.

### 20.5.2 Your current indexes
You added some indexes already to Article and Task. Good.

### 20.5.3 How to tell if you need an index
Signs:
- query slows down as table grows
- `explain()` shows sequential scan on large table
- filtering by a field is common and slow

### 20.5.4 Use `.explain()` to inspect query plans
In shell:

```python
qs = Task.objects.filter(organization=org, status="open").order_by("-created_at")
print(qs.explain())
```

In PostgreSQL, you’ll see whether index is used. SQLite explain is less rich but still useful.

**Industry note:** indexing strategy matters most in PostgreSQL, but learning the habit now helps.

---

## 20.6 Pagination: Protect Performance and UX

### 20.6.1 Why pagination is mandatory
Unbounded lists:
- grow without limit
- slow DB and responses
- slow templates and browsers
- increase memory usage

### 20.6.2 Offset pagination vs cursor pagination (scaling limit reminder)
- Offset: simple (`?page=2`), but slow at huge offsets and can shift results.
- Cursor: stable and faster at scale but more complex.

For most “blog/task list” apps, offset pagination is acceptable initially.

---

## 20.7 Caching: The Biggest “Multipliers” When Used Correctly

Caching is about storing computed results so you don’t recompute every request.

In Django, common cache layers:

1. per-view caching (cache entire response)
2. template fragment caching (cache part of a page)
3. low-level caching (cache computed data in code)

### 20.7.1 Cache backends (conceptual)
- local memory (not shared across processes; dev only)
- Redis/Memcached (production standard)
- database cache (less common)

We’ll start with local memory cache to learn mechanics.

---

## 20.8 Configure a Simple Cache Backend (Development Learning Setup)

In `config/settings.py` (dev):

```python
CACHES = {
    "default": {
        "BACKEND": "django.core.cache.backends.locmem.LocMemCache",
        "LOCATION": "django-mastery-cache",
    }
}
```

Explanation:
- locmem cache stores values in memory of the Django process.
- In production with multiple workers, locmem won’t share cache across workers.
- But it’s perfect for learning.

---

## 20.9 Per-View Caching (Cache an Entire Page)

If your article list is mostly public and changes infrequently, you can cache it.

### 20.9.1 Cache the articles list for 60 seconds
In `articles/views.py`:

```python
from django.views.decorators.cache import cache_page

@cache_page(60)
def article_list(request):
    ...
```

#### Explanation
- The entire response for a given URL (including query string) is cached for 60 seconds.
- That means:
  - `/articles/?tag=django` is a different cache entry than `/articles/?tag=cbv`.

#### Important warning (auth-related)
Do not cache pages that include user-specific content unless you vary cache keys by user or avoid caching.

Your article list is public. That’s a good candidate.

---

## 20.10 Template Fragment Caching (Cache Just the Expensive Part)

Suppose your sidebar shows “Top tags” computed by an annotated query. You can cache that fragment.

### 20.10.1 Add “top tags” query in view
In `articles/views.py`:

```python
from django.db.models import Count, Q

top_tags = (
    Tag.objects.annotate(
        published_count=Count(
            "articles",
            filter=Q(articles__status=Article.Status.PUBLISHED),
            distinct=True,
        )
    )
    .order_by("-published_count", "name")[:10]
)
```

Pass it to template:

```python
return render(..., {"top_tags": top_tags, ...})
```

### 20.10.2 Cache fragment in template
In your template:

```django
{% load cache %}

{% cache 300 top_tags %}
  <aside>
    <h3>Top tags</h3>
    <ul>
      {% for t in top_tags %}
        <li>
          <a href="{% url 'articles:list' %}?tag={{ t.slug }}">{{ t.name }}</a>
          ({{ t.published_count }})
        </li>
      {% endfor %}
    </ul>
  </aside>
{% endcache %}
```

#### Explanation: what the `{% cache %}` tag does
- caches rendered HTML of that block for 300 seconds
- key is `top_tags` (plus template context variation rules)
- reduces DB work and template rendering work

#### Common mistake
If the fragment depends on user-specific data, you must include user-specific keys. For public lists, it’s fine.

---

## 20.11 Low-Level Caching (Cache Computed Values in Code)

This is for caching data like:
- “top tags” query result
- site settings
- expensive calculations

Example:

```python
from django.core.cache import cache

def get_top_tags():
    key = "top_tags_v1"
    data = cache.get(key)
    if data is not None:
        return data

    data = list(Tag.objects.order_by("name")[:10].values("name", "slug"))
    cache.set(key, data, timeout=300)
    return data
```

Explanation:
- `cache.get` returns cached value or None
- compute if missing
- store with TTL
- return cached data next time

### Cache invalidation (the hard part)
When tags/articles change, cache might become stale.
Strategies:
- short TTL (simple)
- explicit cache key bump (`top_tags_v2`)
- delete cache key on save signals (more complex)
- use versioned caching with last-updated timestamps

Start with short TTL unless you truly need real-time correctness.

---

## 20.12 Avoid Common Performance Traps in Templates

### 20.12.1 Don’t call expensive ORM operations inside loops
Bad:

```django
{% for a in articles %}
  {{ a.tags.count }}
{% endfor %}
```

This may run a query per article (depending on prefetch).

Better:
- prefetch tags and render them
- annotate counts in queryset if you need counts

### 20.12.2 Avoid heavy filters inside loops
Template filters like `truncatechars` are cheap; complex custom filters might not be.
If you do heavy computations, do them in view/service and pass results.

---

## 20.13 DB Transaction and Locking Performance (Basics)
Transactions can impact concurrency:
- long transactions lock rows and block others
- keep transactions short in views
- avoid doing network calls inside `atomic()` blocks

This becomes critical in payments, inventory, etc.

---

# 20.14 LAB: Optimize Two Hot Paths (Articles List + Tasks List)

## Lab A — Articles list optimization
1. Ensure `article_list` uses:
   - `.select_related("author")`
   - `.prefetch_related("tags")`
2. Use Django Debug Toolbar to verify:
   - queries are stable (avoid per-article tag queries)
3. Add query-count regression test.

## Lab B — Tasks list optimization
1. Ensure `task_list` uses:
   - `.select_related("assigned_to", "created_by", "updated_by")`
2. Verify query count doesn’t grow with number of tasks.
3. Add a query-count test for task list view.

## Lab C — Add caching
1. Add locmem cache.
2. Cache article list view for 30–60 seconds.
3. Add a “Top tags” fragment cache for 5 minutes.
4. Compare request timing header before/after.

---

## 20.15 Performance Regression Test Examples

### Django TestCase `assertNumQueries`
Example for tasks list:

```python
from django.test import TestCase
from django.urls import reverse

from orgs.models import Membership
from tests.factories import MembershipFactory, OrganizationFactory, TaskFactory, UserFactory


class TaskListQueryTests(TestCase):
    def test_task_list_query_count_stable(self):
        org = OrganizationFactory(slug="acme")
        user = UserFactory(username="u1")
        MembershipFactory(organization=org, user=user, role=Membership.Role.MEMBER)

        # Create many tasks
        for _ in range(30):
            TaskFactory(organization=org)

        self.client.login(username="u1", password="pass12345")

        url = reverse("tasks:list", kwargs={"org_slug": "acme"})

        with self.assertNumQueries(5):
            response = self.client.get(url)
            self.assertEqual(response.status_code, 200)
```

The exact number depends on middleware/auth/session. The key is:
- it should not become 35 queries when tasks count is 30.

---

## 20.16 Production Scaling Notes (What Comes Later)

As your app grows, scaling usually proceeds:

1. Optimize ORM + indexes
2. Add caching (Redis)
3. Add async/background tasks for slow operations (email, exports)
4. Use CDNs for static/media
5. Scale DB (read replicas, connection pooling)
6. Add monitoring/alerts

Django can scale extremely well when you handle the above correctly.

---

## 20.17 Exercises (Do These Before Proceeding)

1. Use Debug Toolbar to find one N+1 and fix it with prefetch/select_related.
2. Add an index to a field you filter frequently (e.g., `Task.status` with org), make migrations, and explain what query it improves.
3. Add `cache_page(60)` to one safe public view and confirm the response time drops on second request.
4. Add one fragment cache in a template and explain what it caches and when it becomes stale.
5. Write one `assertNumQueries` test that fails before your optimization and passes after.

---

## 20.18 Chapter Summary (What you should retain)

- Performance work is measure → fix → verify.
- N+1 queries are the biggest Django performance killer; fix with select_related/prefetch_related.
- Indexes make filtering and ordering fast at scale.
- Pagination prevents unbounded work.
- Caching (view/fragment/low-level) multiplies performance when used carefully.
- Performance regressions should be caught with query-count tests and monitoring.

---

Next chapter: **Part IV — 21. Logging, Monitoring, and Observability**  
We’ll implement production-grade logging (structured logs, request IDs), integrate
error tracking concepts, add health checks and metrics ideas, and create an
observability checklist suitable for real deployments.

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='19. security.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='21. logging_monitoring_scaling_foundations.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
